In [ ]:
import numpy as np
from scipy import spatial
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
import pandas as pd
import re
from scipy.spatial import distance

In [ ]:
# if mounting on google drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
# download glove embeddings with spec 840b 300d
embeddings_dict = {}
error = []
with open("glove.840B/glove.840B.300d.txt") as f:
    for line in f:
        values = line.split()
        word = values[0]
        try:
          vector = np.asarray(values[1:], "float32")
          embeddings_dict[word] = vector
        except:
          error.append(word)

In [ ]:
# https://github.com/adimaini/WEAT-WEFAT/blob/main/src/lib/weat.py
import numpy as np
import itertools
from scipy import stats
from scipy.stats.stats import zscore
import statistics

class Weat:

    def cos_similarity(self, tar, att):
        '''
        Calculates the cosine similarity of the target variable vs the attribute
        '''
        score = np.dot(tar, att) / (np.linalg.norm(tar) * np.linalg.norm(att))
        return score


    def mean_cos_similarity(self, tar, att):
        '''
        Calculates the mean of the cosine similarity between the target and the range of attributes
        '''
        mean_cos = np.mean([self.cos_similarity(tar, attribute) for attribute in att])
        return mean_cos


    def association(self, tar, att1, att2):
        '''
        Calculates the mean association between a single target and all of the attributes
        '''
        association = self.mean_cos_similarity(tar, att1) - self.mean_cos_similarity(tar, att2)
        return association

    def differential_association(self, t1, t2, att1, att2):
        '''
        xyz
        '''
        diff_association = np.sum([self.association(tar1, att1, att2) for tar1 in t1]) - \
                        np.sum([self.association(tar2, att1, att2) for tar2 in t2])
        return diff_association


    def effect_size(self, t1, t2, att1, att2):
        '''
        Calculates the effect size (d) between the two target variables and the attributes
        Parameters:
            t1 (np.array): first target variable matrix
            t2 (np.array): second target variable matrix
            att1 (np.array): first attribute variable matrix
            att2 (np.array): second attribute variable matrix

        Returns:
            effect_size (float): The effect size, d.

        Example:
            t1 (np.array): Matrix of word embeddings for professions "Programmer, Scientist, Engineer"
            t2 (np.array): Matrix of word embeddings for professions "Nurse, Librarian, Teacher"
            att1 (np.array): matrix of word embeddings for males (man, husband, male, etc)
            att2 (np.array): matrix of word embeddings for females (woman, wife, female, etc)
        '''
        combined = np.concatenate([t1, t2])
        num1 = np.mean([self.association(target, att1, att2) for target in t1])
        num2 = np.mean([self.association(target, att1, att2) for target in t2])
        combined_association = np.array([self.association(target, att1, att2) for target in combined])
        dof = combined_association.shape[0]
        denom = np.sqrt(((dof-1)*np.std(combined_association, ddof=1) ** 2 ) / (dof-1))
        effect_size = (num1 - num2) / denom
        return effect_size



    def p_value(self, t1, t2, att1, att2):
        '''
        calculates the p value associated with the weat test
        '''
        diff_association = self.differential_association(t1, t2, att1, att2)
        target_words = np.concatenate([t1, t2])
        np.random.shuffle(target_words)

        # check if join of t1 and t2 have even number of elements, if not, remove last element
        if target_words.shape[0] % 2 != 0:
            target_words = target_words[:-1]

        partition_differentiation = []
        for i in range(10000):
            seq = np.random.permutation(target_words)
            tar1_words = seq[:len(target_words) // 2]
            tar2_words = seq[len(target_words) // 2:]
            partition_differentiation.append(
                self.differential_association(tar1_words, tar2_words, att1, att2)
                )

        mean = np.mean(partition_differentiation)
        stdev = np.std(partition_differentiation)
        p_val = 1 - stats.norm(loc=mean, scale=stdev).cdf(diff_association)

        # print("Mean: ", mean, "\n\n", "stdev: ", stdev, "\n\n partition ass: ", partition_differentiation, '\n\n association: ', diff_association, '\n\n p value: ', p_val)
        return p_val, diff_association, partition_differentiation

class Wefat(Weat):

    def effect_size(self, tar, att1, att2):
        '''
        Calculates the effect size (d) between the target variable vector and the attributes

        Parameters:
            tar (np.array):  target variable vector
            att1 (np.array): first attribute variable matrix
            att2 (np.array): second attribute variable matrix

        Returns:
            effect_size (float): The effect size, d.

        Example:
            tar (np.array): Vector of word embeddings for a profession "Programmer"
            att1 (np.array): matrix of word embeddings for males (man, husband, male, etc)
            att2 (np.array): matrix of word embeddings for females (woman, wife, female, etc)
        '''
        if len(tar)==300: # check to ensure that it is a vector, and not a matrix
            combined = np.concatenate([att1, att2])
            num = self.association(tar, att1, att2)
            cos_similarities = np.array([self.cos_similarity(tar, att) for att in combined])
            dof = cos_similarities.shape[0]
            denom = np.sqrt(((dof-1)*np.std(cos_similarities, ddof=1) **2 ) / (dof-1))
            effect_size = num / denom
            return effect_size
        else:
            raise ValueError("Passed array is not a vector, but a matrix")

    def p_value(self, tar, att1, att2):
        '''
        calculates the p-value associated with the wefat test
        '''
        association = self.association(tar, att1, att2)
        attributes = np.concatenate([att1, att2])
        np.random.shuffle(attributes)

        # check if join of t1 and t2 have even number of elements, if not, remove last element
        if attributes.shape[0] % 2 != 0:
            attributes = attributes[:-1]

        partition_association = []
        for i in range(1000000):
            seq = np.random.permutation(attributes)
            att1_words = seq[:len(attributes) // 2]
            att2_words = seq[len(attributes) // 2:]
            partition_association.append(
                self.association(tar, att1_words, att2_words)
                )

        mean = np.mean(partition_association)
        stdev = np.std(partition_association)
        p_val = 1 - stats.norm(loc=mean, scale=stdev).cdf(association)
        # print("Mean: ", mean, "\n\n", "stdev: ", stdev, "\n\n partition ass: ", partition_association, '\n\n association: ', association, '\n\n p value: ', p_val)
        return p_val, association, partition_association

<ipython-input-9-bce3d4c2f507>:5: DeprecationWarning: Please use `zscore` from the `scipy.stats` namespace, the `scipy.stats.stats` namespace is deprecated.
  from scipy.stats.stats import zscore


In [ ]:
# family of gender attributes
female_attributes = ["female", "woman", "girl", "sister", "she", "her", "hers", "daughter"]
male_attributes = ["male", "man", "boy", "brother", "he", "him", "his", "son"]

In [ ]:
# embeddings
female_embed = np.array([embeddings_dict[i] for i in female_attributes])
male_embed = np.array([embeddings_dict[i] for i in male_attributes])

In [ ]:
female = pd.read_csv("female_model_generations.csv")
male = pd.read_csv("male_model_generations.csv")
neutral = pd.read_csv("neutral_model_generations.csv")
prompts = pd.concat([female, male, neutral])

In [ ]:
female_to_male = {"woman":"man", "father": "mother", "girl": "boy", "mother": "father", "boy":"girl", "man":"woman", "she": "he", "husband":"wife", "daughter": "son", "uncle":"aunt", "brother":"sister", "nephew":"niece",
"female":"male", "grandmother":'grandfather', "grandson": "granddaughter", "girlfriend": "boyfriends", "fatherinlaw":"sisterinlaw", "lady":"gentleman", "him":"her"}
male_to_female = {v:k for k,v in female_to_male.items()}

In [ ]:
# isolate career and gender attributes from instances
# the following two code blocks assumes you are working with DYNAMIC PROMPT 2
prompts = female

regex = re.compile('[^a-zA-Z]')
prompts_list = list(prompts['Task'])
stimuli = []
for i in prompts_list:
  pattern = r"[‘’'](\w+)[‘’']"
  s = re.findall(pattern, i)
  if len(s) == 3:
    stimuli.append(s)
  else:
    try:
      if s[0] in female_to_male:
        s.append(female_to_male[s[0]])
        stimuli.append(s)
      elif s[0] in male_to_female:
        s.append(male_to_female[s[0]])
        stimuli.append(s)
    except:
      continue

In [ ]:
new_df = pd.DataFrame()
weat = Wefat()
for i in range(len(stimuli)):
  if (stimuli[i][0] not in embeddings_dict):
    print(stimuli[i][0])
  elif (stimuli[i][1] not in embeddings_dict):
    print(stimuli[i][1])
  else:
    target_embedding = embeddings_dict[stimuli[i][1]]
    row = dict(prompts.iloc[i])
    effect_size = weat.effect_size(target_embedding, female_embed, male_embed)

    row['effect_size'] = effect_size
    new_df = pd.concat([new_df, pd.DataFrame(row, index=[i])], ignore_index=True)
new_df.to_csv("neutral_gen_weat_scores.csv")